In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn import metrics
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('/home/intermittent-renewables-production-france.csv')
df.head(1)

In [ ]:
# Os seguintes blocos realizam os tratamentos necessários dos dados (Deletando e 'dummyzando')
df1 = df.copy()

df1 = df1.drop(['Date and Hour',	'Date',	'StartHour',	'EndHour'], axis=1)
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.dropna(inplace=True)

In [ ]:
df1.isnull().sum()

In [ ]:
print(df1['dayName'].value_counts())
print(df1['monthName'].value_counts())
print(df1['dayOfYear'].value_counts())

In [ ]:
df1['Source'] = df.apply(lambda row: 1 if row['Source'] == 'Solar' else 0, axis=1)
df1['Wednesday'] = df.apply(lambda row: 1 if row['dayName'] == 'Wednesday' else 0, axis=1)
df1['Thursday'] = df.apply(lambda row: 1 if row['dayName'] == 'Thursday' else 0, axis=1)
df1['Friday'] = df.apply(lambda row: 1 if row['dayName'] == 'Friday' else 0, axis=1)
df1['Saturday'] = df.apply(lambda row: 1 if row['dayName'] == 'Saturday' else 0, axis=1)
df1['Monday'] = df.apply(lambda row: 1 if row['dayName'] == 'Monday' else 0, axis=1)
df1['Tuesday'] = df.apply(lambda row: 1 if row['dayName'] == 'Tuesday' else 0, axis=1)
df1['Sunday'] = df.apply(lambda row: 1 if row['dayName'] == 'Sunday' else 0, axis=1)

df1['January'] = df.apply(lambda row: 1 if row['monthName'] == 'January' else 0, axis=1)
df1['February'] = df.apply(lambda row: 1 if row['monthName'] == 'February' else 0, axis=1)
df1['March'] = df.apply(lambda row: 1 if row['monthName'] == 'March' else 0, axis=1)
df1['April'] = df.apply(lambda row: 1 if row['monthName'] == 'April' else 0, axis=1)
df1['May'] = df.apply(lambda row: 1 if row['monthName'] == 'May' else 0, axis=1)
df1['June'] = df.apply(lambda row: 1 if row['monthName'] == 'June' else 0, axis=1)
df1['July'] = df.apply(lambda row: 1 if row['monthName'] == 'July' else 0, axis=1)
df1['August'] = df.apply(lambda row: 1 if row['monthName'] == 'August' else 0, axis=1)
df1['September'] = df.apply(lambda row: 1 if row['monthName'] == 'September' else 0, axis=1)
df1['October'] = df.apply(lambda row: 1 if row['monthName'] == 'October' else 0, axis=1)
df1['November'] = df.apply(lambda row: 1 if row['monthName'] == 'November' else 0, axis=1)
df1['December'] = df.apply(lambda row: 1 if row['monthName'] == 'December' else 0, axis=1)

df1.drop(['dayName', 'monthName'], axis=1, inplace=True)
df1.head(2)

In [ ]:
df1['Source'].value_counts()

In [ ]:
df_solar = df1.copy()  # Define qual será o df para o Source 'Solar'
df_wind = df1.copy()   # Define qual será o df para o Source 'Wind'

In [ ]:
# Separa as bases entre o 'df_solar' e 'df_wind'
df_solar = df_solar[df_solar['Source'] != 0]
df_wind = df_wind[df_wind['Source'] != 1]

In [ ]:
df_solar['Source'].value_counts()  # Verificação se foi feito corretamente

In [ ]:
# Apenas calcula o percentil de 33 e 66 da coluna 'Production' e realiza a criação das classes da variável solicitada dentro de 'df_solar'

percentil_production33 = df_solar['Production'].quantile(0.33)
percentil_production66 = df_solar['Production'].quantile(0.66)
df_solar['Production'] = df_solar.apply(lambda row: 0 if row['Production'] < percentil_production33 else
                          2 if row['Production'] > percentil_production66 else 1, axis=1)
df_solar["Production"].value_counts()

In [ ]:
print(df_solar['Production'].value_counts())  # Printa o tamanho das classes

In [ ]:
# balanceia as amostras em 'df_solar'
df_production2 = df_solar[df_solar['Production']==2]
df_production1 = df_solar[df_solar['Production']==1]
df_production0 = df_solar[df_solar['Production']==0]
print(df_production2['Production'].value_counts())
print(df_production1['Production'].value_counts())
print(df_production0['Production'].value_counts())

In [ ]:
df_production2 = df_production2.sample(n=9633)
df_production1 = df_production1.sample(n=9633)
df_production0 = df_production0.sample(n=9633)
df_solar = pd.concat([df_production2, df_production1, df_production0])
df_solar['Production'].value_counts()

In [ ]:
# Apenas calcula o percentil de 33 e 66 da coluna 'Production' e realiza a criação das classes da variável solicitada dentro de 'df_wind'

percentil_production33 = df_wind['Production'].quantile(0.33)
percentil_production66 = df_wind['Production'].quantile(0.66)
df_wind['Production'] = df_wind.apply(lambda row: 0 if row['Production'] < percentil_production33 else
                          2 if row['Production'] > percentil_production66 else 1, axis=1)
df_wind["Production"].value_counts()

In [ ]:
print(df_wind['Production'].value_counts())  # Printa o tamanho das classes

In [ ]:
# balanceia as amostras em 'df_wind'
df_production2 = df_wind[df_wind['Production']==2]
df_production1 = df_wind[df_wind['Production']==1]
df_production0 = df_wind[df_wind['Production']==0]
print(df_production2['Production'].value_counts())
print(df_production1['Production'].value_counts())
print(df_production0['Production'].value_counts())

In [ ]:
df_production2 = df_production2.sample(n=9866)
df_production1 = df_production1.sample(n=9866)
df_production0 = df_production0.sample(n=9866)
df_wind = pd.concat([df_production2, df_production1, df_production0])
df_wind['Production'].value_counts()

In [ ]:
# O seguinte algoritmo realiza o calculo da Regressão Logística para a base de teste de 'df_solar'

x = df_solar.drop('Production', axis='columns')
y = df_solar['Production']

x_scaled = preprocessing.StandardScaler().fit(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5)

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print(f"Acuracia", accuracy_score(y_test, y_pred))

In [ ]:
# O seguinte algoritmo realiza o calculo da Regressão Logística para a base de teste de 'df_wind'

x = df_wind.drop('Production', axis='columns')
y = df_wind['Production']

x_scaled = preprocessing.StandardScaler().fit(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5)

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print(f"Acuracia", accuracy_score(y_test, y_pred))

In [ ]:
print(df_solar.columns)  # Recupera a lista de valores para adicionar em nossa variavel 'x'

In [ ]:
# O seguinte algoritmo cria uma rede Multi Layer Perceptron Classifier (MLP Classifier) para o dataset 'df_solar', utilizando os
# hiperparametros solicitados e apresentando a acuracia de treino e de teste
def MLPClassifier_evaluate(df_solar, features, target):
  x = df_solar[features]
  y = df_solar[[target]]
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

  ss = StandardScaler()
  x_train_scaled = ss.fit_transform(x_train)
  x_test_scaled = ss.transform(x_test)

  nn = MLPClassifier(
      max_iter=5000,
       hidden_layer_sizes=(6,6,6,6),
      activation='tanh',
      learning_rate= 'constant',
      random_state=42)
  nn.fit(x_train_scaled, y_train.values.ravel())

  y_test_pred_solar = nn.predict(x_test_scaled)
  y_train_pred_solar = nn.predict(x_train_scaled)

  print("Acuracia treino:", accuracy_score(y_train, y_train_pred_solar))
  print("Acuracia teste:", accuracy_score(y_test, y_test_pred_solar))



features = (['Source', 'dayOfYear', 'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Monday', 'Tuesday', 'Sunday', 'January', 'February',
       'March', 'April', 'May', 'June', 'July', 'August', 'September',
       'October', 'November', 'December'])
target = 'Production'
MLPClassifier_evaluate(df_solar, features, target)

In [ ]:
# O seguinte algoritmo cria uma rede Multi Layer Perceptron Classifier (MLP Classifier) para o dataset 'df_wind', utilizando os
# hiperparametros solicitados e apresentando a acuracia de treino e de teste
def MLPClassifier_evaluate(df_wind, features, target):
  x = df_wind[features]
  y = df_wind[[target]]
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

  ss = StandardScaler()
  x_train_scaled = ss.fit_transform(x_train)
  x_test_scaled = ss.transform(x_test)

  nn = MLPClassifier(
      max_iter=5000,
       hidden_layer_sizes=(6,6,6,6),
      activation='tanh',
      learning_rate= 'constant',
      random_state=42)
  nn.fit(x_train_scaled, y_train.values.ravel())

  y_test_pred_nn = nn.predict(x_test_scaled)
  y_train_pred_nn = nn.predict(x_train_scaled)

  print("Acuracia treino:", accuracy_score(y_train, y_train_pred_nn))
  print("Acuracia teste:", accuracy_score(y_test, y_test_pred_nn))


features = (['Source', 'dayOfYear', 'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Monday', 'Tuesday', 'Sunday', 'January', 'February',
       'March', 'April', 'May', 'June', 'July', 'August', 'September',
       'October', 'November', 'December'])
target = 'Production'
MLPClassifier_evaluate(df_wind, features, target)